In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

In [25]:
df = pd.read_csv('archivo_unificado.csv')
df.head()

,FechaTransaccion,IdTransaccion,IdCliente,CdCadena,IdCiudad,SUM_of_VlrVentasConImpuestos
0,25MAR2021,4851730852,5104794.0,C,11001,5.3061
1,02APR2021,4857458053,5434724.0,C,11001,4.5000
2,19FEB2021,4828227464,1844294.0,C,11001,8.8889
3,19MAR2021,4848040431,29966990.0,C,47001,68.2250
4,16APR2021,4865257354,9500366.0,C,11001,198.1602


In [26]:
df.describe()

,IdTransaccion,IdCliente,IdCiudad,SUM_of_VlrVentasConImpuestos
count,6.987576e+07,6.987576e+07,6.987576e+07,6.987576e+07
mean,5.180701e+09,2.691689e+07,1.562939e+04,3.700558e+01
std,2.257328e+08,4.326381e+07,1.660928e+04,8.735992e+01
min,4.800820e+09,1.400000e+01,5.001000e+03,1.000000e-04
25%,4.981618e+09,3.619066e+06,1.100100e+04,5.476100e+00
50%,5.173503e+09,7.221785e+06,1.100100e+04,1.455550e+01
75%,5.378833e+09,2.709722e+07,1.100100e+04,3.776890e+01
max,5.576283e+09,1.523072e+08,7.689200e+04,3.561842e+04


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69875755 entries, 0 to 69875754
Data columns (total 6 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   FechaTransaccion              object 
 1   IdTransaccion                 int64  
 2   IdCliente                     float64
 3   CdCadena                      object 
 4   IdCiudad                      int64  
 5   SUM_of_VlrVentasConImpuestos  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 3.1+ GB


In [28]:
nulos_id_cliente = df['IdCliente'].isnull().sum()
print(f"El número de valores nulos en la columna 'IdCliente' es: {nulos_id_cliente}")
nulos_fecha = df['FechaTransaccion'].isnull().sum()
print(f"El número de valores nulos en la columna 'FechaTransaccion' es: {nulos_fecha}")
transaccion = df['IdTransaccion'].isnull().sum()
print(f"El número de valores nulos en la columna 'ValorTransaccion' es: {transaccion}")
ciudad = df['IdCiudad'].isnull().sum()
print(f"El número de valores nulos en la columna 'IdCiudad' es: {ciudad}")
valor = df['SUM_of_VlrVentasConImpuestos'].isnull().sum()
print(f"El número de valores nulos en la columna 'SUM_of_VlrVentasConImpuestos' es: {valor}")

El número de valores nulos en la columna 'IdCliente' es: 0
El número de valores nulos en la columna 'FechaTransaccion' es: 0
El número de valores nulos en la columna 'ValorTransaccion' es: 0
El número de valores nulos en la columna 'IdCiudad' es: 0
El número de valores nulos en la columna 'SUM_of_VlrVentasConImpuestos' es: 0


In [29]:
df['FechaTransaccion'] = pd.to_datetime(
    df['FechaTransaccion'], format='%d%b%Y')

In [30]:
# Calcular los ingresos por transacción
df['IngresosPorTransaccion'] = df['SUM_of_VlrVentasConImpuestos']

In [31]:
# Calcular los ingresos por cliente
df['IngresosPorCliente'] = df.groupby(
    'IdCliente')['IngresosPorTransaccion'].transform('sum')

In [32]:
# Calcular el beneficio por cliente (en este caso, es igual a los ingresos por cliente)
df['BeneficioPorCliente'] = df['IngresosPorCliente']

In [33]:

# Calcular el CLV
tasa_descuento = 0.1  # Ajusta la tasa de descuento según tus necesidades
clv = df['BeneficioPorCliente'].sum() / (1 + tasa_descuento)
print(f"El Customer Lifetime Value (CLV) es: {clv}")

El Customer Lifetime Value (CLV) es: 928172356973.886


In [34]:
print(f"El Customer Lifetime Value (CLV) es: {clv}")

El Customer Lifetime Value (CLV) es: 928172356973.886



El valor que has obtenido para el Customer Lifetime Value (CLV) es de 
928,172,356,973.886
928,172,356,973.886. Este valor representa la estimación del valor total que se espera obtener de todos los clientes a lo largo de su relación con la empresa, teniendo en cuenta los ingresos generados por las transacciones.

Recuerda que el CLV es una métrica importante, pero su interpretación puede variar según el contexto y la industria. Además, es fundamental considerar otros factores y métricas para obtener una visión más completa de la salud financiera y la eficacia de las estrategias de retención de clientes.







In [38]:
# Calcular Recency
today = datetime.today()
df['Recency'] = (today - df.groupby('IdCliente')
                 ['FechaTransaccion'].transform('max')).dt.days

# Características para el modelo
# Ajusta según tus columnas disponibles
features = ['Recency', 'SUM_of_VlrVentasConImpuestos']

# Variable objetivo (BeneficioPorCliente en este caso)
# Asumiendo que ya tienes el BeneficioPorCliente calculado
target = 'BeneficioPorCliente'

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    df[features], df[target], test_size=0.2, random_state=42)

# Inicializar el modelo de Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')